In [1]:
import keras.backend as K
import tensorflow as tf

Using TensorFlow backend.


In [78]:
import numpy as np
anchors       = np.array(((0.8224980405412367, 0.8422009361658516),
                         (0.3996183458965756, 0.7135638022627926),
                         (0.3124547970691455, 0.356975225778714),
                         (0.715922252069278, 0.47470965095772916),
                         (0.1272335914117044, 0.19679513461111448)))

n_anchors = len(anchors)
n_classes = 20
batch     = 2

In [104]:
# Example output layer of YOLOv2 in Batch
y_pred      = K.random_uniform_variable(shape=(batch, 3, 4, n_anchors, n_classes+5),low=0, high=1)

# Examples of ground truth values
label_idx = np.random.randint(0, n_classes)
y = np.random.randint(128,size=(batch, n_classes+5)).astype(np.float)
y[:,4] = 1 # set objectiveness score
y[:,5:]= 0 # set one_hot
y[:,5+label_idx]= 1
print(y[0])
y_true = np.tile(y, (1, 4*3*n_anchors)).reshape([batch, 4*3, n_anchors, n_classes+5])
y_true = K.variable(y_true)
# Anchors
anchors_tensor = K.reshape(K.variable(anchors), [1, 1, 1, n_anchors, 2])

[  71.   26.    5.  110.    1.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    1.    0.    0.    0.    0.    0.    0.    0.
    0.]


In [105]:
tf.InteractiveSession()
tf.global_variables_initializer().run()

In [106]:
print("Ground truth boxes: [batch_size, grid_w*grid_h, box_params]")
print(y_true.eval().shape)
print("Predicted boxes: [batch_size, height, width, n_anchors, 5 + n_classes]")
print(y_pred.eval().shape)

Ground truth boxes: [batch_size, grid_w*grid_h, box_params]
(2, 12, 5, 25)
Predicted boxes: [batch_size, height, width, n_anchors, 5 + n_classes]
(2, 3, 4, 5, 25)


In [107]:
pred_shape = K.shape(y_pred)[1:3]

GRID_W = tf.cast(pred_shape[0], tf.float32)
GRID_H = tf.cast(pred_shape[1], tf.float32)
NORM_W = tf.cast(32.*GRID_W, tf.float32)
NORM_H = tf.cast(32.*GRID_H, tf.float32)
N_CLASSES = n_classes
CLASS      = n_classes
ANCHORS   = anchors
SCALE_NOOB = 0.5
SCALE_CONF = 5.0
SCALE_COOR = 5.0
SCALE_PROB = 1.0

BOX              = len(ANCHORS)
N_ANCHORS        = BOX

print(NORM_W.eval())

96.0


In [122]:
# Create Grid map of output layer
cx = K.arange(0, stop=pred_shape[0])
cx = K.tile(cx, [pred_shape[1]])
cy = K.arange(0, stop=pred_shape[1])
cy = K.tile(K.expand_dims(cy, 0), [pred_shape[0], 1])
cy = K.flatten(K.transpose(cy))

c_xy = K.transpose(K.stack([cx, cy]))
c_xy = K.reshape(c_xy, [1, pred_shape[1], pred_shape[0], 2])
c_xy = tf.transpose(c_xy, [0, 2, 1, 3])
c_xy = K.cast(c_xy, K.dtype(y_pred))
c_xy = K.expand_dims(c_xy, 3)
c_xy = K.tile(c_xy, [1, 1, 1, N_ANCHORS, 1])

In [123]:
y_pred = K.reshape(y_pred, [-1, pred_shape[0], pred_shape[1], N_ANCHORS, N_CLASSES + 5])
y_true = K.reshape(y_true, [-1, pred_shape[0], pred_shape[1], N_ANCHORS, N_CLASSES + 5])
anchor_tensor = K.reshape(ANCHORS, [1, 1, 1, N_ANCHORS, 2])

y_pred_shape = K.shape(y_pred)
### Adjust prediction
# adjust x and y # adjust w and h     
pred_box_xy = tf.sigmoid(y_pred[:,:,:,:,:2])+ c_xy
pred_box_wh   = tf.exp(y_pred[:,:,:,:,2:4]) * K.cast(anchor_tensor, dtype=K.dtype(y_pred))
pred_box_wh   = tf.sqrt(pred_box_wh / K.reshape([GRID_W, GRID_H], [1,1,1,1,2]))
pred_box_conf = tf.expand_dims(tf.sigmoid(y_pred[:, :, :, :, 4]), -1) # adjust confidence: {P_objxIoU}
pred_box_prob = tf.nn.softmax(y_pred[:, :, :, :, 5:]) # adjust probability

print(K.sigmoid(y_pred[...,:2]).eval().shape) # [x, y] predction
print("Adjusted Predicted Centroid of Bounding Box")
print(pred_box_xy.eval()[0][1][2])
print("Adjusted Predicted With/Height of Bounding Box")
print(pred_box_wh.eval()[0][1][2])

(2, 3, 4, 5, 2)
Adjusted Predicted Centroid of Bounding Box
[[ 1.71716034  2.58981609]
 [ 1.58380818  2.62486482]
 [ 1.63618958  2.51320934]
 [ 1.71979415  2.63625646]
 [ 1.66058481  2.59275889]]
Adjusted Predicted With/Height of Bounding Box
[[ 0.58237809  0.58765244]
 [ 0.45490813  0.42399994]
 [ 0.41613922  0.37682661]
 [ 0.52141809  0.48469809]
 [ 0.2658411   0.32954934]]


In [124]:
### Adjust ground truth
# adjust x, y, w ,h
true_box_xy = y_true[:,:,:,:,0:2] / K.reshape([(NORM_W/GRID_W), (NORM_H/GRID_H)], [1,1,1,1,2])
true_box_wh = tf.sqrt(y_true[:,:,:,:,2:4]  / K.reshape([NORM_W, NORM_H], [1,1,1,1,2]))

print(y_true[...,:2].eval().shape) # [x, y] predction
print("Adjusted Ground truth Centroid of Bounding Box")
print(true_box_xy.eval()[0][1][2])
print("Adjustes Ground truth With/Height of Bounding Box [Relative]")
print(true_box_wh.eval()[0][1][2])

(2, 3, 4, 5, 2)
Adjusted Ground truth Centroid of Bounding Box
[[ 2.21875  0.8125 ]
 [ 2.21875  0.8125 ]
 [ 2.21875  0.8125 ]
 [ 2.21875  0.8125 ]
 [ 2.21875  0.8125 ]]
Adjustes Ground truth With/Height of Bounding Box [Relative]
[[ 0.22821774  0.92702478]
 [ 0.22821774  0.92702478]
 [ 0.22821774  0.92702478]
 [ 0.22821774  0.92702478]
 [ 0.22821774  0.92702478]]


In [99]:
# adjust confidence
pred_tem_wh = tf.pow(pred_box_wh, 2) * K.reshape([GRID_W, GRID_H], [1,1,1,1,2])
pred_box_area = pred_tem_wh[:,:,:,:,0] * pred_tem_wh[:,:,:,:,1]
pred_box_ul = pred_box_xy - 0.5 * pred_tem_wh
pred_box_bd = pred_box_xy + 0.5 * pred_tem_wh

true_tem_wh = tf.pow(true_box_wh, 2) * K.reshape([GRID_W, GRID_H], [1,1,1,1,2])
true_box_area = true_tem_wh[:,:,:,:,0] * true_tem_wh[:,:,:,:,1]
true_box_ul = true_box_xy - 0.5 * true_tem_wh
true_box_bd = true_box_xy + 0.5 * true_tem_wh

intersect_ul = tf.maximum(pred_box_ul, true_box_ul) 
intersect_br = tf.minimum(pred_box_bd, true_box_bd)
intersect_wh = intersect_br - intersect_ul
intersect_wh = tf.maximum(intersect_wh, 0.0)
intersect_area = intersect_wh[:,:,:,:,0] * intersect_wh[:,:,:,:,1]

iou = tf.truediv(intersect_area, true_box_area + pred_box_area - intersect_area)
detected_objects = K.cast(iou > 0.3, K.dtype(iou))

print(iou.eval().shape)
print("IoU GT & Prediction")
print(iou.eval()[0])

print("\n\nAdjusted IoU > certain threshold")
print(detected_objects.eval().shape)
print(detected_objects.eval()[0])

(2, 3, 4, 5)
IoU GT & Prediction
[[[ 0.          0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.          0.        ]]

 [[ 0.          0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.          0.        ]
  [ 0.08456285  0.00805055  0.04277421  0.02820491  0.        ]
  [ 0.25879475  0.09616462  0.03391831  0.09069596  0.0105485 ]]

 [[ 0.          0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.          0.        ]
  [ 0.28087306  0.05945022  0.02553074  0.13938642  0.01349546]
  [ 0.46574184  0.27338773  0.04353723  0.26121804  0.02363139]]]


Adjusted IoU > certain threshold
(2, 3, 4, 5)
[[[ 0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.]]

 [[ 0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.]

In [100]:
object_scale      = 5
no_object_scale   = 1
class_scale       = 1
coordinates_scale = 1

y_true = tf.concat([true_box_xy, true_box_wh, true_box_conf, true_box_prob], 4)
y_pred = tf.concat([pred_box_xy, pred_box_wh, pred_box_conf, pred_box_prob], 4)

# Objectivess Loss
no_object_weights = no_object_scale * (1 - detected_objects) *(1 - detectors_mask)
no_objects_loss   = no_object_weights * K.square(-pred_box_conf)
objects_loss      = object_scale * detectors_mask * K.square(1 - pred_confidence)

confidence_loss = objects_loss + no_objects_loss

# Localization loss
coor_loss = coordinates_scale*tf.nn.l2_loss(pred_box_xy - true_box_xy)
# Objectiveness Loss

# Classification loss - cross-entropy
clf_loss =  tf.nn.softmax_cross_entropy_with_logits(logits=pred_box_prob, labels=true_box_prob)*best_box

# print(coor_loss.eval())
print(clf_loss.eval())

NameError: name 'detectors_mask' is not defined